In [52]:
import os
import json

BASEDIR='../model_discovery/model/library/base'

def read_reports(baseline):
    dir = os.path.join(BASEDIR, baseline,'reports')
    reports = {}
    for file in os.listdir(dir):
        if file.startswith('report_'):
            scale = file.split('.')[0].split('_')[1]
            with open(os.path.join(dir, file), 'r') as f:
                reports[scale]=json.load(f)
    return reports

def read_eval_reports(baseline):
    report=read_reports(baseline)
    eval_reports={}
    for i in report:
        eval_reports[i]=report[i]['eval_results.json']
    return eval_reports

def read_eval_results(baseline):
    reports=read_eval_reports(baseline)
    results={}
    for i in reports:
        results[i]=reports[i]['results']
    return results

BASELINES = ['gpt2','mamba2','ttt','retnet','rwkv6','RANDOM']

gpt2_results=read_eval_results('gpt2')
random_results=read_eval_results('RANDOM')['0']

#### Print datasets and metrics

In [63]:
random_results
metrics={}
for i in random_results:
    res=random_results[i]
    alias=res['alias']
    for m in res:
        if m=='alias': continue
        if '_stderr' in m: continue
        if m not in metrics:
            metrics[m]=[]
        metrics[m].append(alias)
for m in metrics:
    print(m,metrics[m])

{'acc,none': ['arc_challenge', 'arc_easy', 'blimp_adjunct_island_filtered', 'blimp_anaphor_gender_agreement_filtered', 'blimp_anaphor_number_agreement_filtered', 'blimp_animate_subject_passive_filtered', 'blimp_animate_subject_trans_filtered', 'blimp_causative_filtered', 'blimp_complex_NP_island_filtered', 'blimp_coordinate_structure_constraint_complex_left_branch_filtered', 'blimp_coordinate_structure_constraint_object_extraction_filtered', 'blimp_determiner_noun_agreement_1_filtered', 'blimp_determiner_noun_agreement_2_filtered', 'blimp_determiner_noun_agreement_irregular_1_filtered', 'blimp_determiner_noun_agreement_irregular_2_filtered', 'blimp_determiner_noun_agreement_with_adj_2_filtered', 'blimp_determiner_noun_agreement_with_adj_irregular_1_filtered', 'blimp_determiner_noun_agreement_with_adj_irregular_2_filtered', 'blimp_determiner_noun_agreement_with_adjective_1_filtered', 'blimp_distractor_agreement_relational_noun_filtered', 'blimp_distractor_agreement_relative_clause_filte

In [60]:


def readout_results(results,metric):
    out={}
    for i in results:
        res=results[i]
        alias=res['alias']
        if metric in res:
            out[alias]=res[metric]
    return out

def readout_allresults(allresults,metric):
    out={}
    for scale in allresults:
        results=allresults[scale]
        out[scale]=readout_results(results,metric)
    return out

def avg_results(results,keyword=None):
    if keyword:
        results=filter_results(results,keyword)
    avg=0
    for i in results:
        avg+=results[i]
    return avg/len(results)

def filter_results(results,keyword):
    out={}
    for i in results:
        if keyword in i:
            out[i]=results[i]
    return out

gpt2_accs = readout_allresults(gpt2_results,'acc,none')
random_acc = readout_results(random_results,'acc,none')

In [62]:
for scale in gpt2_accs:
    print(f'GPT2 {scale}:',avg_results(gpt2_accs[scale],'blimp'))
print('Random:',avg_results(random_acc,'blimp'))

GPT2 70M: 0.6546198903678867
GPT2 14M: 0.6804744525703669
GPT2 31M: 0.7142609187492691
Random: 0.5899860624855434
